In [4]:
import os
import pandas as pd

In [5]:
df=pd.DataFrame()
for folder in os.listdir('results'):
    try:
        df=pd.concat([df,pd.read_csv(r'results/'+folder+'/results_kpi.csv')])
    except:
        print('No `results_kpi.csv` in ' + folder)
df.sort_values('p_pv')
df.to_csv('results.csv')

In [6]:
df['battery_discharge [kWh]'] / df['battery_charge [kWh]']

0    0.757437
dtype: float64

In [22]:
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
import numpy as np
df=pd.read_csv('results.csv')
df=df.loc[df['p_pv']>0.1,:]
df=df.loc[df['e_bat']!=0.1,:]

In [43]:
from matplotlib.pyplot import ticklabel_format


colorscale=[
        [0, "rgb(183,0,0)"],
        [0.1, "rgb(183,0,0)"],
        [0.1, "rgb(255,56,0)"],
        [0.2, "rgb(255,56,0)"],
        [0.2, "rgb(255,90,0)"],
        [0.3, "rgb(255,90,0)"],
        [0.3, "rgb(255,146,0)"],
        [0.4, "rgb(255,146,0)"],
        [0.4, "rgb(255,203,0)"],
        [0.5, "rgb(255,203,0)"],
        [0.5, "rgb(195,255,0)"],
        [0.6, "rgb(195,255,0)"],
        [0.6, "rgb(153,204,51)"],
        [0.7, "rgb(153,204,51)"],
        [0.7, "rgb(67,166,12)"],
        [0.8, "rgb(67,166,12)"],
        [0.8, "rgb(51,153,0)"],
        [0.9, "rgb(51,153,0)"],
        [0.9, "rgb(38,98,54)"],
        [1, "rgb(38,98,54)"],]
colorbar=dict(tickmode='array',tickvals=[0, 0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95],ticktext=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90','90-100'],len=1.05,ticklabeloverflow='allow')

In [44]:
fig= make_subplots(1,2)
fig.add_trace(go.Heatmap(x=df['p_pv'].values,
                                y=df['e_bat'].values,
                                z=df['autarkiegrad'].values,
                                text=df['autarkiegrad'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis'
                                
),1,1)
fig.add_trace(go.Heatmap(x=df['p_pv'].values,
                                y=df['e_bat'].values,
                                z=df['eigenverbrauch'].values,
                                text=df['eigenverbrauch'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis',
                                opacity=1,                                                       
),1,2)
fig.update_layout(title='Autarkiegrad und Eigenverbrauch',width=900.5,height=500,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
fig.update_yaxes(tick0=0,dtick=0.25,tickangle=270,title='C_eff_sp in kWh/MWh')
fig.update_xaxes(tick0=0,dtick=0.25,title='p_pv_sp in kWp/MWh')
fig.show()
